# Investment Planning for Electricity Generation

*Daniel Cortild, Anastasia Bouwsma, Esteban Castillo Mondragón*

## 1

In [1]:
from scipy.optimize import linprog
import numpy as np
import pandas as pd
import cvxpy as cp
import time
import gurobipy

print(cp.installed_solvers())
def print_solution(problem,x):
    print(f"Objective value: {problem.value}")
    print(f"Capacity of each technology: {x.value[0:n]}")
    precost=sum([c[i]*x.value[i] for i in range(n)])
    prodcost=sum([c[i]*x.value[i] for i in np.arange(n,len(x.value))])
    print(f"Investment costs for phase 1: {precost}")
    print(f"Production costs for phase 2: {prodcost}")
n=4
k=3    

['CLARABEL', 'ECOS', 'ECOS_BB', 'GUROBI', 'OSQP', 'SCIPY', 'SCS']


### Expected Value Problem

In [2]:

c = [10.,7.,16.,6.]  #costs for instalation
c=c 
#costs
c= c+[v*i for v in [10,6,1]   for i in [4,4.5,3.2,5.5]]

#phase 1 constraints
#energy requirement constraign
Aub = [[-1]*n+[0]*(len(c)-n)]
bub=[-7-4-3]
#budget
Aub=Aub+[c[0:n]+[0]*(len(c)-n)]
bub.append(120)

#phase 2 constraints
#capacity
Aub = Aub +[[0]*i+[-1]+[0]*(k-i)+(([0]*i+[1]+[0]*(k-i))*k) for i in range(n)]
bub = bub+ [0]*n

#power requirement
Aub = Aub + [[0]*(n+n*i)+[-1]*n+[0]*(len(c)-8-n*i) for i in range(k)]
bub = bub + [-5,-3,-2]

x=cp.Variable(len(c))
Aub = np.array(Aub)
c= np.array(c)
bub = np.array(bub)
restricts=[Aub @ x <= bub]+ [x >= 0]
problem = cp.Problem(cp.Minimize(c.T @x),restricts )

res= problem.solve(solver = cp.GUROBI)
avres=list(x.value[0:n])
precost=sum([c[i]*x.value[i] for i in range(n)])
prodcost=sum([c[i]*x.value[i] for i in np.arange(n,len(x.value))])
print_solution(problem,x)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-10-03
Objective value: 394.6666666666667
Capacity of each technology: [2.83333333 3.         2.16666667 6.        ]
Investment costs for phase 1: 120.0
Production costs for phase 2: 274.6666666666667


### Expected Value Solution

In [3]:
#function to test the average sollution in practice
#c = [10.,7.,16.,6.]  #costs for instalation
#costs
demands = [[z,j,i] for i in [-1,-2,-3] for j in [-2,-3,-4] for z in  [-3,-5,-7] ]
probs = [z*j*i for i in [0.3,0.4,0.3] for j in [0.3,0.4,0.3] for z in [0.3,0.4,0.3]]

c= [v*i*p for p in probs for v in [10,6,1]   for i in [4,4.5,3.2,5.5] ]#add cost of usage

#phase 1 constraints
#energy requirement constraign
#Aub = [[-1]*4+[0]*(len(c)-4)]
#bub=[-7-4-3]
#budget
#Aub=Aub+[c[0:4]+[0]*(len(c)-4)]
#bub.append(120)

#phase 2 constraints
#capacity
Aub =[[0]*(12*j)+(([0]*i+[1]+[0]*(3-i))*3)+[0]*(len(c)-12-12*j ) for j in range(len(probs)) for i in range(n) ]
bub =  avres*len(probs)

#power requirement
Aub = Aub + [[0]*(4*i+12*j)+[-1]*4+[0]*(len(c)-4-4*i-12*j) for j in range(len(probs)) for i in range(k)]

bub = bub +[i  for t in range(len(demands)) for i in demands[t] ] 

x=cp.Variable(len(c))
Aub = np.array(Aub)
c= np.array(c)
bub = np.array(bub)
restricts=[Aub @ x <= bub]+ [x >= 0]
problem = cp.Problem(cp.Minimize(c.T @x),restricts )

res3= problem.solve(solver = cp.GUROBI)
print(res3+precost)

399.5926666666666


### Regular Problem

In [6]:
c = [10.,7.,16.,6.]  #costs for instalation
#costs
demands = [[z,j,i] for i in [-1,-2,-3] for j in [-2,-3,-4] for z in  [-3,-5,-7] ]
probs = [z*j*i for i in [0.3,0.4,0.3] for j in [0.3,0.4,0.3] for z in [0.3,0.4,0.3]]

c= c+[v*i*p for p in probs for v in [10,6,1]   for i in [4,4.5,3.2,5.5] ]#add cost of usage

#phase 1 constraints
#energy requirement constraign
Aub = [[-1]*4+[0]*(len(c)-4)]
bub=[-7-4-3]
#budget
Aub=Aub+[c[0:4]+[0]*(len(c)-4)]
bub.append(120)

#phase 2 constraints
#capacity
Aub = Aub +[[0]*(i )+[-1]+[0]*(3-i+12*k)+(([0]*i+[1]+[0]*(3-i))*3)+[0]*(len(c)-16-12*k ) for k in range(len(probs)) for i in range(4) ]
bub = bub+ [0]*4*len(probs)

#power requirement
Aub = Aub + [[0]*(4+4*i+12*k)+[-1]*4+[0]*(len(c)-8-4*i-12*k) for k in range(len(probs)) for i in range(3)]

bub = bub +[i  for t in range(len(demands)) for i in demands[t] ] 


x=cp.Variable(len(c))
Aub = np.array(Aub)
c= np.array(c)
bub = np.array(bub)
restricts=[Aub @ x <= bub]+ [x >= 0]
problem = cp.Problem(cp.Minimize(c.T @x),restricts )

res2= problem.solve(solver = cp.GUROBI)
print_solution(problem,x)

Objective value: 397.75133333333326
Capacity of each technology: [3.16666667 5.         1.83333333 4.        ]
Investment costs for phase 1: 120.0
Production costs for phase 2: 277.7513333333333


### Perfect Information

In [7]:
demands = [[z,j,i] for i in [-1,-2,-3] for j in [-2,-3,-4] for z in  [-3,-5,-7] ]
probs = [z*j*i for i in [0.3,0.4,0.3] for j in [0.3,0.4,0.3] for z in [0.3,0.4,0.3]]
c = [10.,7.,16.,6.]  #costs for instalation
#costs
c= c+[v*i for v in [10,6,1]   for i in [4,4.5,3.2,5.5]]

#phase 1 constraints
#energy requirement constraign
Aub = [[-1]*4+[0]*(len(c)-4)]
bub=[-7-4-3]
#budget
Aub=Aub+[c[0:4]+[0]*(len(c)-4)]
bub.append(120)

#phase 2 constraints
#capacity
Aub = Aub +[[0]*i+[-1]+[0]*(3-i)+(([0]*i+[1]+[0]*(3-i))*3) for i in range(n)]
bub = bub+ [0]*n

#power requirement
Aub = Aub + [[0]*(4+4*i)+[-1]*4+[0]*(len(c)-8-4*i) for i in range(k)]

costs = []
for a in range(len(demands)):
    #costs = costs+[linprog(c=c, A_ub = Aub, b_ub = bub+(demands[a])).fun*probs[a]]
    #print(sum(costs))  
    x=cp.Variable(len(c))
    Aub = np.array(Aub)
    c= np.array(c)
    b_ub_temp = np.array(bub+demands[a])
    restricts=[Aub @ x <= b_ub_temp]+ [x >= 0]
    problem = cp.Problem(cp.Minimize(c.T @x),restricts )

    costs=costs+[problem.solve(solver = cp.GUROBI)*probs[a]]
print(f"Total costs: {sum(costs)}")

Total costs: 394.9666666666667
